In [ ]:
!pip install gdown
!gdown 1WdZIp6oTyB9Ug7ZkttsXcC0mtpquZ0_x -O hymenoptera.zip

Downloading...
From (original): https://drive.google.com/uc?id=1WdZIp6oTyB9Ug7ZkttsXcC0mtpquZ0_x
From (redirected): https://drive.google.com/uc?id=1WdZIp6oTyB9Ug7ZkttsXcC0mtpquZ0_x&confirm=t&uuid=231ce194-f343-4df1-85c6-a3f5cda6ceb6
To: /content/hymenoptera.zip
100% 47.3M/47.3M [00:01<00:00, 37.8MB/s]


In [ ]:
!unzip hymenoptera.zip -d /content

Archive:  hymenoptera.zip
   creating: /content/hymenoptera_data/
   creating: /content/hymenoptera_data/train/
   creating: /content/hymenoptera_data/train/ants/
  inflating: /content/hymenoptera_data/train/ants/0013035.jpg  
  inflating: /content/hymenoptera_data/train/ants/1030023514_aad5c608f9.jpg  
  inflating: /content/hymenoptera_data/train/ants/1095476100_3906d8afde.jpg  
  inflating: /content/hymenoptera_data/train/ants/1099452230_d1949d3250.jpg  
  inflating: /content/hymenoptera_data/train/ants/116570827_e9c126745d.jpg  
  inflating: /content/hymenoptera_data/train/ants/1225872729_6f0856588f.jpg  
  inflating: /content/hymenoptera_data/train/ants/1262877379_64fcada201.jpg  
  inflating: /content/hymenoptera_data/train/ants/1269756697_0bce92cdab.jpg  
  inflating: /content/hymenoptera_data/train/ants/1286984635_5119e80de1.jpg  
  inflating: /content/hymenoptera_data/train/ants/132478121_2a430adea2.jpg  
  inflating: /content/hymenoptera_data/train/ants/1360291657_dc248c5eea.j

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [ ]:
# Load the CIFAR-10 dataset
(x_cifar10_train, y_cifar10_train), (x_cifar10_test, y_cifar10_test) = cifar10.load_data()

# Normalize pixel values
x_cifar10_train = x_cifar10_train.astype('float32') / 255
x_cifar10_test = x_cifar10_test.astype('float32') / 255

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [ ]:
# Load the Hymenoptera value
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_hymenoptera(path, target_size=(32,32)):
    X, Y = [], []
    for label, folder in enumerate(['ants','bees']):  # ants=0, bees=1 initially
        files = os.listdir(os.path.join(path, folder))
        for f in files:
            img = load_img(os.path.join(path, folder, f), target_size=target_size)
            img_arr = img_to_array(img)
            X.append(img_arr)
            Y.append(label)
    return np.array(X), np.array(Y)

x_hymen_train, y_hymen_train = load_hymenoptera('hymenoptera_data/train', target_size=(32,32))
x_hymen_test, y_hymen_test = load_hymenoptera('hymenoptera_data/val', target_size=(32,32))

# Normalize pixel values
x_hymen_train = x_hymen_train.astype('float32') / 255
x_hymen_test = x_hymen_test.astype('float32') / 255

Idea of label: cifar10 labels 0-9, hymen labels 10, 11.

So y_hymen += 10 (from 0,1 to 10,11)

In [ ]:
y_hymen_train += 10
y_hymen_test += 10

In [ ]:
# Concat
x_train = np.concatenate([x_cifar10_train, x_hymen_train], axis=0)
y_train = np.concatenate([y_cifar10_train.flatten(), y_hymen_train], axis=0)

x_test = np.concatenate([x_cifar10_test, x_hymen_test], axis=0)
y_test = np.concatenate([y_cifar10_test.flatten(), y_hymen_test], axis=0)

In [ ]:
model = Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(12, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=15, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/15
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.2630 - loss: 1.9879 - val_accuracy: 0.5055 - val_loss: 1.3923
Epoch 2/15
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.5212 - loss: 1.3605 - val_accuracy: 0.5950 - val_loss: 1.1582
Epoch 3/15
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.5945 - loss: 1.1763 - val_accuracy: 0.6223 - val_loss: 1.0678
Epoch 4/15
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.6378 - loss: 1.0436 - val_accuracy: 0.6503 - val_loss: 1.0030
Epoch 5/15
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.6695 - loss: 0.9621 - val_accuracy: 0.6702 - val_loss: 0.9582
Epoch 6/15
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.6915 - loss: 0.8928 - val_accuracy: 0.6809 - val_loss: 0.9255
Epoch 7/15
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.7139 - loss: 0.8296 - val_accuracy: 0.6774 - val_loss: 0.9463
Epoch 8/15
1571/1571 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.7336 - loss: 0.7727 -

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
test_acc *= 100
print(f'Test accuracy: {test_acc:.2f}%')

318/318 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7123 - loss: 0.9144
Test accuracy: 70.87%
